# Automatic-Data-Cleaning# Automatic Data Cleaning & Analysis Agent

## 🎯 Objective

This project implements an **Automatic Data Cleaning & Analysis Agent** that:
- Loads raw **CSV/Excel** datasets
- Automatically **profiles**, **cleans**, and **analyzes** the data
- Exports:
  - A **cleaned dataset** ready for modeling

## 🧩 Problem

Data scientists and analysts spend a lot of time:
- Inspecting dataset structure (shape, columns, dtypes)
- Handling missing values and duplicates
- Fixing inconsistent types
- Detecting simple outliers
- Writing the same boilerplate code again and again

This preprocessing phase is:
- Time-consuming
- Error-prone
- Repeated in every new project

## ✅ Proposed Solution

Build an **agent-style system** that:

1. **Loads** any CSV/Excel file
2. **Analyzes** the structure:
   - Shape, columns, data types
   - Missing value report
   - Duplicates
3. **Cleans** the data:
   - Handles missing values (per column type)
   - Detects & removes duplicates
   - Fixes inconsistent types
   - Detects basic outliers

In [16]:
import os
import pandas as pd
import asyncio
import tempfile
from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters # Required for defining server parameters
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types


In [9]:
import nest_asyncio
nest_asyncio.apply()
print("✅ Applied nest-asyncio patch for Notebook environment.")

✅ Applied nest-asyncio patch for Notebook environment.


In [25]:


def load_sample(file_path: str, n: int = 5):
    """
    Loads a sample of the dataset.
    Args:
        file_path: path to csv
        n: number of rows to show (integer)
    """
    # Force n to be an integer in case the LLM sends 5.0
    n_int = int(n) 
    df = pd.read_csv(file_path)
    return df.head(n_int).to_string()

def dataset_summary(path: str) -> dict:
    """Basic dataset info."""
    df = pd.read_csv(path)
    return {
        "rows": len(df),
        "columns": list(df.columns),
        "dtypes": df.dtypes.astype(str).to_dict()
    }



In [11]:

def load_sample(path: str, n: int = 100) -> str:
    """Load a random sample of the dataset."""
    df = pd.read_csv(path)
    return df.sample(min(n, len(df))).to_csv(index=False)


def dataset_summary(path: str) -> dict:
    """Basic dataset info."""
    df = pd.read_csv(path)
    return {
        "rows": len(df),
        "columns": list(df.columns),
        "dtypes": df.dtypes.astype(str).to_dict()
    }


In [12]:

def missing_report(path: str) -> dict:
    df = pd.read_csv(path)
    return (df.isnull().mean() * 100).to_dict()

def duplicate_count(path: str) -> int:
    df = pd.read_csv(path)
    return int(df.duplicated().sum())


In [35]:
def handle_missing_median(path: str, output: str) -> str:
    """Impute missing numeric values with median per column."""
    df = pd.read_csv(path)
    df = df.fillna(df.median(numeric_only=True))
    df.to_csv(output, index=False)
    return f"Missing values imputed with median. Saved to {output}"

def handle_missing_mode(path: str, output: str) -> str:
    """Impute missing categorical values with mode per column."""
    df = pd.read_csv(path)
    for col in df.select_dtypes(include=['object', 'category']):
        df[col] = df[col].fillna(df[col].mode()[0] if not df[col].mode().empty else 'Unknown')
    df.to_csv(output, index=False)
    return f"Missing values imputed with mode. Saved to {output}"

def handle_missing_ffill(path: str, output: str) -> str:
    """Forward-fill missing values."""
    df = pd.read_csv(path)
    df = df.fillna(method='ffill')
    df.to_csv(output, index=False)
    return f"Missing values forward-filled. Saved to {output}"


def detect_outliers_iqr(path: str) -> dict:
    """Detect outliers using IQR method. Returns column-wise outlier counts."""
    df = pd.read_csv(path)
    outliers = {}
    for col in df.select_dtypes(include=[np.number]):
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers[col] = ((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))).sum()
    return outliers

def remove_outliers_iqr(path: str, output: str) -> str:
    """Remove outliers using IQR method."""
    df = pd.read_csv(path)
    for col in df.select_dtypes(include=[np.number]):
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        df = df[~((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR)))]
    df.to_csv(output, index=False)
    return f"Outliers removed via IQR. Saved to {output}"

def fix_data_types(path: str, output: str) -> str:
    """Attempt to convert columns to appropriate types (e.g., dates, numbers)."""
    df = pd.read_csv(path)
    for col in df.columns:
        if df[col].dtype == 'object':
            try:
                df[col] = pd.to_datetime(df[col])
            except:
                try:
                    df[col] = pd.to_numeric(df[col])
                except:
                    pass  # Leave as object
    df.to_csv(output, index=False)
    return f"Data types fixed where possible. Saved to {output}"

def correlation_matrix(path: str) -> str:
    """Compute and return correlation matrix for numeric columns."""
    df = pd.read_csv(path)
    corr = df.corr(numeric_only=True)
    return corr.to_string()

def suggest_features(path: str) -> list:
    """Suggest potentially useful features based on correlations (>0.5)."""
    df = pd.read_csv(path)
    corr = df.corr(numeric_only=True)
    suggestions = []
    for i in range(len(corr.columns)):
        for j in range(i+1, len(corr.columns)):
            if abs(corr.iloc[i, j]) > 0.5:
                suggestions.append(f"High correlation between {corr.columns[i]} and {corr.columns[j]}")
    return suggestions

def detailed_summary(path: str) -> dict:
    """Extended summary: shape, dtypes, stats, unique values."""
    df = pd.read_csv(path)
    return {
        "shape": df.shape,
        "dtypes": df.dtypes.astype(str).to_dict(),
        "describe": df.describe(include='all').to_dict(),
        "unique_counts": df.nunique().to_dict()
    }
    
    
def encode_categorical(path: str, output: str) -> str:
    """Label-encode categorical columns."""
    df = pd.read_csv(path)
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    for col in df.select_dtypes(include=['object']):
        df[col] = le.fit_transform(df[col].astype(str))
    df.to_csv(output, index=False)
    return f"Categorical columns encoded. Saved to {output}"

def standardize_numeric(path: str, output: str) -> str:
    """Standardize numeric columns (z-score)."""
    df = pd.read_csv(path)
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
    df.to_csv(output, index=False)
    return f"Numeric columns standardized. Saved to {output}"


    



In [13]:

def remove_duplicates(path: str, output: str) -> str:
    df = pd.read_csv(path)
    df = df.drop_duplicates()
    df.to_csv(output, index=False)
    return f"Duplicates removed. Saved to {output}"

def handle_missing_mean(path: str, output: str) -> str:
    df = pd.read_csv(path)
    df = df.fillna(df.mean(numeric_only=True))
    df.to_csv(output, index=False)
    return f"Missing values handled. Saved to {output}"


In [14]:

def validate_dataset(path: str) -> dict:
    df = pd.read_csv(path)
    return {
        "rows": len(df),
        "missing_rate": (df.isnull().mean() * 100).to_dict()
    }


In [32]:
import google.generativeai as genai

cleaning_agent = genai.GenerativeModel(
    model_name="gemini-pro",
    tools=[
        load_sample,
        dataset_summary,
        missing_report,
        duplicate_count,
        remove_duplicates,
        handle_missing_mean,
        validate_dataset
    ],
    system_instruction="""
    You are a data cleaning expert.
    1. Inspect the dataset thoroughly (use detailed_summary, correlation_matrix).
    2. Detect issues: missing values, duplicates, outliers, type inconsistencies, correlations.
    3. Prioritize: Remove duplicates first, then handle missing values (choose method based on column type/skew), fix types, detect/remove outliers if numeric, encode categoricals if needed.
    4. For large datasets (>10k rows), sample first (use load_sample).
    5. Validate after each step (use validate_dataset).
    6. If correlations suggest redundancy, suggest feature removal.
    7. Always save intermediate/cleaned versions to avoid data loss.
Never modify data directly — use tools. Handle errors gracefully (e.g., skip invalid columns).
    """
)


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()  # load variables from .env

my_secret_key = os.getenv("GOOGLE_API_KEY")

# 2. Authenticate Gemini
import google.generativeai as genai
genai.configure(api_key=my_secret_key)

c:\Users\dell\Desktop\projects\Automatic-Data-Cleaning-Analysis-Agent\agent_env\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
c:\Users\dell\Desktop\projects\Automatic-Data-Cleaning-Analysis-Agent\agent_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import pandas as pd

In [ ]:


# 2. Define your tool list
# Note: Ensure your functions have clear docstrings so Gemini understands how to use them.
data_tools = [
    load_sample,
    dataset_summary,
    missing_report,
    duplicate_count,
    remove_duplicates,
    handle_missing_median,
    validate_dataset,
    standardize_numeric,
    encode_categorical,
    detailed_summary,
    correlation_matrix,
    suggest_features,
    handle_missing_mode,
    handle_missing_ffill,
    detect_outliers_iqr,
    remove_outliers_iqr,
    fix_data_types
]

# 3. Initialize the Model with Tools
# We use 'gemini-1.5-flash' or 'gemini-1.5-pro' for better tool-calling support
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash", 
    tools=data_tools,
    system_instruction="""
    You are a data cleaning expert.
    1. Inspect the dataset thoroughly (use detailed_summary, correlation_matrix).
    2. Detect issues: missing values, duplicates, outliers, type inconsistencies, correlations.
    3. Prioritize: Remove duplicates first, then handle missing values (choose method based on column type/skew), fix types, detect/remove outliers if numeric, encode categoricals if needed.
    4. For large datasets (>10k rows), sample first (use load_sample).
    5. Validate after each step (use validate_dataset).
    6. If correlations suggest redundancy, suggest feature removal.
    7. Always save intermediate/cleaned versions to avoid data loss.
Never modify data directly — use tools. Handle errors gracefully (e.g., skip invalid columns).
    """
)


In [44]:
import time
from google.api_core.exceptions import ResourceExhausted

# Assuming model is already configured
max_retries = 3
for attempt in range(max_retries):
    try:
        chat = model.start_chat(enable_automatic_function_calling=True)
        
        # 5. Execute the task
        response = chat.send_message("""
        Clean the dataset located at: data/banks-reviews-raw.csv

        Steps:
        - Inspect the data
        - Remove duplicates
        - Handle missing numeric values
        - Validate the cleaned dataset
        - Save final version to data/clean_banks-reviews-raw.csv
        """)

        print(response.text)
        break  # Success, exit loop
    except ResourceExhausted as e:
        print(f"Quota exceeded: {e}")
        if attempt < max_retries - 1:
            time.sleep(32)  # Wait before retry
        else:
            print("Max retries reached. Exiting.")

NotFound: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.